In [1]:
# This code writes the product data table to a TXT file.

product_data = [
    ["Metric", "Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"],
    ["Annual demand (units)", "10,000", "15,000", "25,000", "2,000", "1,500", "95,000"],
    ["Order cost ($)", "50", "50", "50", "50", "50", "150"],
    ["Price/unit load ($)", "500", "650", "350", "250", "225", "150"],
    ["Space required (m²)", "10", "15", "25", "10", "12", "13"],
    ["Reserve dwell percentage (%)", "0", "0", "20%", "0", "0", "100%"],
    ["Yearly carrying cost rate (%)", "10%", "10%", "10%", "10%", "10%", "10%"]
]

with open("product_data.txt", "w") as f:
    for row in product_data:
        f.write("\t".join(row) + "\n")

In [7]:
# This code writes the flow/product data table to a TXT file.

flow_product_data = [
    ["Flow/Product", "Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"],
    ["Flow 1 (CD)", "0.0707", "0.0203", "0.0267", "0.3354", "0.4083", "0.0726"],
    ["Flow 2 (R)", "0.0849", "0.2023", "0.0420", "0.5590", "0.6804", "0.0871"],
    ["Flow 3 (RF)", "0.1061", "0.2023", "0.0054", "1.0062", "1.2248", "0.1088"],
    ["Flow 4 (F)", "0.0778", "0.2023", "0.0481", "0.0671", "0.8165", "0.0798"]
]

with open("product_cost_data.txt", "w") as f:
    for row in flow_product_data:
        f.write("\t".join(row) + "\n")

In [8]:
# This code writes the new flow/product quantity table to a TXT file.

flow_product_quantity = [
    ["Flow/Product", "Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"],
    ["Flow 1 (CD)", "20", "15", "4", "5", "15", "20"],
    ["Flow 2 (R)", "5", "5", "20", "4", "25", "5"],
    ["Flow 3 (RF)", "10", "10", "1", "5", "45", "10"],
    ["Flow 4 (F)", "15", "10", "9", "1", "30", "15"]
]

with open("product_yearly_cost.txt", "w") as f:
    for row in flow_product_quantity:
        f.write("\t".join(row) + "\n")

In [4]:
# This code writes the functional area bounds table to a TXT file.

functional_area_bounds = [
    ["Functional Area", "Lower bound (m²)", "Upper bound (m²)"],
    ["Cross-docking", "0", "15,000"],
    ["Reserve", "35,000", "75,000"],
    ["Forward", "35,000", "75,000"]
]

with open("functional_area_bounds.txt", "w") as f:
    for row in functional_area_bounds:
        f.write("\t".join(row) + "\n")

In [5]:
# This code writes the levels of space available in each functional area to a TXT file.

functional_area_levels = [
    ["Functional Area", "#Levels"],
    ["Cross-docking", "1"],
    ["Reserve", "1"],
    ["Forward", "1"]
]

with open("functional_area_levels.txt", "w") as f:
    for row in functional_area_levels:
        f.write("\t".join(row) + "\n")